In [ ]:
import os
import tensorflow as tf
import pandas, numpy
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

## Small task: Apply preprocessing
Use sklearn.preprocessing.StandardScaler to normalize the training data before using it for trianing.

Help: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

## Download Dataset

In [ ]:
train_dataset_fp = "../00-datasets/iris.data.csv"
print("Local copy of the dataset file: {}".format(train_dataset_fp))

## Read Dataset using Pandas

In [ ]:
COLUMN_NAMES = ['sepal_len', 'sepal_width', 'petal_len', 'petal_width', 'label']
LABEL_COLUMN = 'label'
LABELS = ["Iris-setosa", "Iris-versicolor", "Iris-virginica"]
df = pandas.read_csv(train_dataset_fp, header=None, names=COLUMN_NAMES, sep=',')

## A bit of explorative analysis

In [ ]:
df.head()

In [ ]:
print(df['label'].value_counts())

In [ ]:
df.describe()

In [ ]:
df['label'] = pandas.Categorical(df['label'])
df['label'] = df.label.cat.codes
plot = plt.scatter(x=df["sepal_len"], y=df["sepal_width"], c=df['label'])
plt.show()

In [ ]:
plot = plt.scatter(x=df["petal_len"], y=df["petal_width"], c=df['label'])
plt.show()

## Build the model

### Some preprocessing

In [ ]:
label = df.pop("label").values.astype(numpy.float)

In [ ]:
one_hot = tf.keras.backend.one_hot(tf.convert_to_tensor(label, dtype=tf.int32), 3)

### Start training

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, one_hot))

In [ ]:
train_dataset = dataset.shuffle(len(df)).batch(10)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])


In [ ]:
history = model.fit(train_dataset, epochs=5)

In [ ]:
# list all data in history
print(history.history.keys())

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc = 'lower right')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc = 'upper right')
plt.show()

In [ ]:
model.save('../04-models/iris/1') 